In [23]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
import joblib
from sklearn.preprocessing import LabelEncoder

In [12]:
#importing data
X=joblib.load('output/X.joblib')
y=joblib.load('output/y.joblib')

In [13]:
print(X.shape,y.shape)

(12798, 40) (12798,)


In [15]:
ncol = X.shape[1]
encoding_dim = 20

In [16]:
input_dim = Input(shape = (ncol, ))

In [17]:
encoded1 = Dense(3000, activation = 'relu')(input_dim)
encoded2 = Dense(2750, activation = 'relu')(encoded1)
encoded3 = Dense(2500, activation = 'relu')(encoded2)
encoded4 = Dense(2250, activation = 'relu')(encoded3)
encoded5 = Dense(2000, activation = 'relu')(encoded4)
encoded6 = Dense(1750, activation = 'relu')(encoded5)
encoded7 = Dense(1500, activation = 'relu')(encoded6)
encoded8 = Dense(1250, activation = 'relu')(encoded7)
encoded9 = Dense(1000, activation = 'relu')(encoded8)
encoded10 = Dense(750, activation = 'relu')(encoded9)
encoded11 = Dense(500, activation = 'relu')(encoded10)
encoded12 = Dense(250, activation = 'relu')(encoded11)
encoded13 = Dense(encoding_dim, activation = 'relu')(encoded12)

# Decoder Layers
decoded1 = Dense(250, activation = 'relu')(encoded13)
decoded2 = Dense(500, activation = 'relu')(decoded1)
decoded3 = Dense(750, activation = 'relu')(decoded2)
decoded4 = Dense(1000, activation = 'relu')(decoded3)
decoded5 = Dense(1250, activation = 'relu')(decoded4)
decoded6 = Dense(1500, activation = 'relu')(decoded5)
decoded7 = Dense(1750, activation = 'relu')(decoded6)
decoded8 = Dense(2000, activation = 'relu')(decoded7)
decoded9 = Dense(2250, activation = 'relu')(decoded8)
decoded10 = Dense(2500, activation = 'relu')(decoded9)
decoded11 = Dense(2750, activation = 'relu')(decoded10)
decoded12 = Dense(3000, activation = 'relu')(decoded11)
decoded13 = Dense(ncol, activation = 'sigmoid')(decoded12)


In [18]:
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded13)

In [19]:
# Compile the Model
autoencoder.compile(optimizer = 'adadelta', loss = 'binary_crossentropy')

In [20]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3000)              123000    
_________________________________________________________________
dense_2 (Dense)              (None, 2750)              8252750   
_________________________________________________________________
dense_3 (Dense)              (None, 2500)              6877500   
_________________________________________________________________
dense_4 (Dense)              (None, 2250)              5627250   
_________________________________________________________________
dense_5 (Dense)              (None, 2000)              4502000   
_________________________________________________________________
dense_6 (Dense)              (None, 1750)              3501

In [ ]:
#label Encoding the data
lb=LabelEncoder()
y=lb.fit_transform(y)

In [28]:
print(y)

[0 0 0 ... 6 6 6]


In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size = 0.7, random_state = seed(2017))

In [33]:
autoencoder.fit(X_train, X_train, nb_epoch = 10, batch_size = 32, shuffle = False, validation_data = (X_test, X_test))

Train on 8958 samples, validate on 3840 samples
Epoch 1/10
  32/8958 [..............................] - ETA: 46s - loss: -500267176883240986013073408.0000

D:\Anacona3\envs\tf_gpuu\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


8958/8958 [==============================] - 48s 5ms/step - loss: -495044019987922596418027520.0000 - val_loss: -498856387728336252022292480.0000
Epoch 2/10
8958/8958 [==============================] - 48s 5ms/step - loss: -501916474044288119647240192.0000 - val_loss: -505698273114891504293249024.0000
Epoch 3/10
8958/8958 [==============================] - 48s 5ms/step - loss: -508720580693991171448373248.0000 - val_loss: -512479830269628055614914560.0000
Epoch 4/10
8958/8958 [==============================] - 48s 5ms/step - loss: -515476520263198753377222656.0000 - val_loss: -519242896100659431733198848.0000
Epoch 5/10
8958/8958 [==============================] - 49s 5ms/step - loss: -522244777274067319762452480.0000 - val_loss: -526024641719631237385027584.0000
Epoch 6/10
8958/8958 [==============================] - 48s 5ms/step - loss: -529032831376802901619900416.0000 - val_loss: -532829619475535299033956352.0000
Epoch 7/10
8958/8958 [==============================] - 48s 5ms/step 

In [34]:
encoder = Model(inputs = input_dim, outputs = encoded13)
encoded_input = Input(shape = (encoding_dim, ))

In [35]:
encoded_train = pd.DataFrame(encoder.predict(X_train))
encoded_train = encoded_train.add_prefix('feature_')

encoded_test = pd.DataFrame(encoder.predict(X_test))
encoded_test = encoded_test.add_prefix('feature_')

In [38]:
encoded_train['target'] = Y_train

In [39]:
print(encoded_train.shape)
encoded_train.head()

(8958, 21)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,2.127306e+11,0.0,0.0,0.0,0.0,1.387671e+13,0.0,0.0,6.592436e+11,2.672479e+12,...,0.0,0.0,3.646861e+12,3.851354e+12,0.0,7.725866e+12,3.143759e+12,2.769225e+12,0.0,3
1,2.119934e+11,0.0,0.0,0.0,0.0,1.320386e+13,0.0,0.0,6.235966e+11,2.552937e+12,...,0.0,0.0,3.465690e+12,3.661130e+12,0.0,7.344184e+12,2.994017e+12,2.640368e+12,0.0,0
2,2.100950e+11,0.0,0.0,0.0,0.0,2.402796e+13,0.0,0.0,1.010358e+12,4.179898e+12,...,0.0,0.0,6.407091e+12,6.570159e+12,0.0,1.338980e+13,5.274323e+12,4.777316e+12,0.0,0
3,1.856446e+11,0.0,0.0,0.0,0.0,1.035958e+13,0.0,0.0,5.224115e+11,2.136997e+12,...,0.0,0.0,2.716453e+12,2.906448e+12,0.0,5.765479e+12,2.412801e+12,2.084038e+12,0.0,1
4,1.938680e+11,0.0,0.0,0.0,0.0,1.129944e+13,0.0,0.0,5.536321e+11,2.276178e+12,...,0.0,0.0,2.965440e+12,3.154844e+12,0.0,6.285114e+12,2.606869e+12,2.270337e+12,0.0,4


In [41]:
encoded_train.to_csv(r'AE_train.csv', index = False)

In [42]:
encoded_test.to_csv(r'AE_test.csv', index = False)

In [43]:
X_train = 'X_train.joblib'
y_train = 'y_train.joblib'
X_test = 'X_test.joblib'
y_test = 'y_test.joblib'
save_dir = 'output/'

savedX = joblib.dump(X, os.path.join(save_dir, X_train))
savedy = joblib.dump(y, os.path.join(save_dir, y_train))
savedX = joblib.dump(X, os.path.join(save_dir, X_test))
savedy = joblib.dump(y, os.path.join(save_dir, y_test))